In [1]:
import sqlite3
import pandas as pd
# inspect current database schema
conn = sqlite3.connect('patient_slim.sqlite')
cursor = conn.cursor()
# list tables
# pull all data from the visits table 
cursor.execute("SELECT * FROM visits;")
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[column[0] for column in cursor.description])
conn.close()

In [2]:
# let's create a new sqlite database called patient_demonstration.sqlite
conn = sqlite3.connect('patient_demonstration.sqlite')
cursor = conn.cursor() 

In [3]:
# create a table called clinical_visits with the column names pulled above
# overwite the table if it already exists
cursor.execute('DROP TABLE IF EXISTS clinical_visits;')
cursor.execute('''
CREATE TABLE clinical_visits (
    PatientPKHash TEXT,
    SiteCode TEXT,
    VisitDate TEXT,
    VisitType TEXT,
    VisitBy TEXT,
    NextAppointmentDate TEXT,
    TCAReason TEXT,
    Pregnant TEXT,
    Breastfeeding TEXT,
    StabilityAssessment TEXT,
    DifferentiatedCare TEXT,
    WHOStage INTEGER,
    WHOStagingOI TEXT,
    Height REAL,
    Weight REAL,    
    EMR TEXT,
    Project TEXT,
    Adherence TEXT,
    AdherenceCategory TEXT,
    BP TEXT,
    OI TEXT,
    OIDate DATE,
    CurrentRegimen TEXT,
    AppointmentReminderWillingness TEXT,
    key TEXT
);
''')

# let's now populate the table with the rows variable that contains all the data from the visits table
cursor.executemany('''
INSERT INTO clinical_visits (PatientPKHash, SiteCode, VisitDate, VisitType, VisitBy, NextAppointmentDate, TCAReason, Pregnant, Breastfeeding, StabilityAssessment, DifferentiatedCare, WHOStage, WHOStagingOI, Height, Weight, EMR, Project, Adherence, AdherenceCategory, BP, OI, OIDate, CurrentRegimen, AppointmentReminderWillingness, key)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
''', rows)
conn.commit()

In [4]:
# now let's create a data dictionary
cursor.execute('DROP TABLE IF EXISTS data_dictionary;')
cursor.execute('''
CREATE TABLE data_dictionary (
    table_name TEXT,
    column_name TEXT,
    description TEXT);
''')

In [5]:
# populate the data dictionary with descriptions for each column in the clinical_visits table
cursor.execute('''
INSERT INTO data_dictionary (table_name, column_name, description) VALUES
('clinical_visits', 'PatientPKHash', 'Hashed patient identifier'),
('clinical_visits', 'SiteCode', 'Code for the clinical site'),
('clinical_visits', 'VisitDate', 'Date of the patient visit'),
('clinical_visits', 'VisitType', 'Type of the patient visit. Values include Unknown, SCHEDULED VISIT, UNSCHEDULED VISIT LATE,
       UNSCHEDULED VISIT EARLY, Unscheduled, Scheduled. These should typically be grouped as Scheduled and Unscheduled'),
('clinical_visits', 'VisitBy', 'Provider of the visit. Values include , Self, Treatment supporter, Refill visit documentation, Other'),
('clinical_visits', 'NextAppointmentDate', 'Date of the next scheduled clinical appointment set during VisitDate. 
               This is typically a date in the future after VisitDate.'),
('clinical_visits', 'TCAReason', 'Reason for the TCA (To Come Again) status. Values include, Follow up, Lab tests, Pharmacy Refill, Counseling,Other'),
('clinical_visits', 'Pregnant', 'Is the patient pregnant? Values include Yes and No.'),
('clinical_visits', 'Breastfeeding', 'Is the patient breastfeeding? Values include Yes, No and N/A'),
('clinical_visits', 'StabilityAssessment', 'Stability assessment result. Values include Stable, Unstable, and not stable.
               typically, this should be grouped as Stable and Unstable (including not stable)'),
('clinical_visits', 'DifferentiatedCare', 'Differentiated care model. Values include Fast Track, Standard Care,
       Community ART Distribution peer led,
       Facility ART distribution Group,
       Community ART Distribution HCW Led'),
('clinical_visits', 'WHOStage', 'WHO stage of the patient, either 1, 2, 3, or 4'),
('clinical_visits', 'WHOStagingOI', 'Opportunistic infection observed during WHO staging. Values include
       Asymptomatic, Oral hairy leukoplakia,
       Unexplained severe weight loss, Pulmonary tuberculosis,
       Extra pulmonary tuberculosis,
       Unexplained severe weight loss,Pulmonary tuberculosis,
       Recurrent upper respiratory tract infections,
       Asymptomatic,Persistent generalized lymphadenopathy),
       Symptomatic HIV-associated nephropathy,
       Cryptococcal meningitis, Herpes zoster,
       Unexplained severe weight loss,Recurrent upper respiratory tract infections,
       Persistent generalized lymphadenopathy),
       Minor mucocutaneous manifestations,
       Unexplained severe weight loss,Unexplained persistent fever,Pulmonary tuberculosis,
       Recurrent oral ulcerations, Unexplained moderate malnutrition,
       Oral candidiasis, HIV wasting syndrome,
       Pulmonary tuberculosis,Oral candidiasis,
       Unexplained persistent fever'),
('clinical_visits', 'Height', 'Height of the patient in centimeters'),
('clinical_visits', 'Weight', 'Weight of the patient in kilograms'),
('clinical_visits', 'EMR', 'Electronic medical record information. Values include AMRS, KenyaEMR, ECARE, DREAMS'),
('clinical_visits', 'Project', 'Project associated with the visit. Values include Ampath Plus, Kenya HMIS II, EDARP, DREAM Kenya Trusts'),
('clinical_visits', 'Adherence', 'Adherence to treatment. Values include Good, , Fair, Good|, Good|Good, Poor, Poor|Poor,
       Poor|, 0, Poor|Good, Good|Poor. This variable will typically be used in combination with AdherenceCategory, and | here should align
               with | in that variable, indicating two values for two categories.'),
('clinical_visits', 'AdherenceCategory', 'Category of adherence. Values include GOOD, , FAIR, ART|CTX, ARV. 
               GOOD and FAIR are erroneous and should be dropped when the variable is used. ART and and ARV should be
               considered as ART.'),
('clinical_visits', 'BP', 'Blood pressure readings. Value reported as systolic/diastolic in mmHg, e.g., 120/80.'),
('clinical_visits', 'OI', 'Opportunistic infections present. Values include Asymptomatic, Lymphadenopathy,
       Respiratory Tract Infections, Moderate Weight Loss'),
('clinical_visits', 'OIDate', 'Date of opportunistic infection diagnosis'),
('clinical_visits', 'CurrentRegimen', 'Current treatment regimen. Value includes two or three digit descriptions of molecules separated by / signs'),
('clinical_visits', 'AppointmentReminderWillingness', 'Willingness to receive appointment reminders. Values include Yes and No'),
('clinical_visits', 'key', 'Unique key for patientPKHash and SiteCode combination');
''')
conn.commit()
conn.close()

In [6]:
conn = sqlite3.connect('patient_slim.sqlite')
cursor = conn.cursor()
# pull all data from the lab table except for the "key" column 
cursor.execute("SELECT * FROM lab;")
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[column[0] for column in cursor.description])
conn.close()

In [7]:
# let's create a new sqlite database called patient_demonstration.sqlite
conn = sqlite3.connect('patient_demonstration.sqlite')
cursor = conn.cursor() 

In [8]:
# create a table called clinical_visits with the column names pulled above
# overwite the table if it already exists
cursor.execute('DROP TABLE IF EXISTS lab;')
cursor.execute('''
CREATE TABLE lab (
    PatientPKHash TEXT,
    SiteCode TEXT,
    OrderedbyDate TEXT,
    ReportedbyDate TEXT,
    TestName TEXT,
    TestResult TEXT,
    key TEXT
);
''')

# let's now populate the table with the rows variable that contains all the data from the visits table
cursor.executemany('''
INSERT INTO lab (PatientPKHash, SiteCode, OrderedbyDate, ReportedbyDate, TestName, TestResult, key)
VALUES (?, ?, ?, ?, ?, ?, ?);
''', rows)
conn.commit()

In [9]:
# now, add lab table to the data dictionary
cursor.execute('''
INSERT INTO data_dictionary (table_name, column_name, description) VALUES
('lab', 'PatientPKHash', 'Hashed patient identifier'),
('lab', 'SiteCode', 'Code for the clinical site'),
('lab', 'OrderedbyDate', 'Date when the lab test was ordered'),
('lab', 'ReportedbyDate', 'Date when the lab test result was reported'),
('lab', 'TestName', 'Name of the lab test conducted, including CD4 Count for adults,
               CD4 Percentage for children, and Viral Load'),
('lab', 'TestResult', 'Result of the lab test. This will sometimes appear as numeric value
               and sometimes as text. Typically, when text, the value will be "LDL", meaning low 
               detectable level, or low HIV viral load.'),
('lab', 'key', 'Unique key for PatientPKHash and SiteCode combination');
''')
conn.commit()
conn.close()  

In [10]:
conn = sqlite3.connect('patient_slim.sqlite')
cursor = conn.cursor()
# pull all data from the lab table except for the "key" column 
cursor.execute("SELECT * FROM pharmacy;")
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[column[0] for column in cursor.description])
conn.close()

In [11]:
# let's create a new sqlite database called patient_demonstration.sqlite
conn = sqlite3.connect('patient_demonstration.sqlite')
cursor = conn.cursor() 

In [12]:
# create a table called clinical_visits with the column names pulled above
# overwite the table if it already exists
cursor.execute('DROP TABLE IF EXISTS pharmacy;')
cursor.execute('''
CREATE TABLE pharmacy (
    PatientPKHash TEXT,
    SiteCode TEXT,
    Drug TEXT,
    DispenseDate TEXT,
    ExpectedReturn TEXT,
    Duration INTEGER,
    TreatmentType TEXT,
    RegimenLine TEXT,
    RegimenChangedSwitched TEXT,
    RegimenChangeSwitchedReason TEXT,
    key TEXT
);
''')

# let's now populate the table with the rows variable that contains all the data from the visits table
cursor.executemany('''
INSERT INTO pharmacy (PatientPKHash, SiteCode, Drug, DispenseDate, ExpectedReturn, Duration, TreatmentType, RegimenLine, RegimenChangedSwitched, RegimenChangeSwitchedReason, key)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
''', rows)
conn.commit()

In [13]:
# now, add pharmacy table to the data dictionary
cursor.execute('''
INSERT INTO data_dictionary (table_name, column_name, description) VALUES
('pharmacy', 'PatientPKHash', 'Hashed patient identifier'),
('pharmacy', 'SiteCode', 'Code for the clinical site'),
('pharmacy', 'Drug', 'Description of the drug prescribed, reported as collection of molecules (e.g. 3TC+DTG+TDF). Most common are ARVs for HIV'),
('pharmacy', 'DispenseDate', 'Date when the drug was dispensed'),
('pharmacy', 'ExpectedReturn', 'Expected return date for the next pharmacy visit'),
('pharmacy', 'Duration', 'Duration in number of days for which the drug is prescribed. Any duration of 60 days or greater is considered a multi-month dispensing (MMD).'),
('pharmacy', 'TreatmentType', 'Type of treatment. Values include ARV, PMTCT, Prophylaxis.'),
('pharmacy', 'RegimenLine', 'Line of treatment regimen. Valid values include First Line, Second Line, Third Line'),
('pharmacy', 'RegimenChangedSwitched', 'Indicates if the regimen was changed or switched. Valid values are Switch and Substition. Otherwise, regimen was not changed.'),
('pharmacy', 'RegimenChangeSwitchedReason', 'Reason for changing or switching the regimen. Valid values include New drug available, Virological failure, Drugs out of stock, Drug toxicity, New Diagnosis of tuberculosis, and Other.'),
('pharmacy', 'key', 'Unique key for PatientPKHash and SiteCode combination');
''')
conn.commit()
conn.close()

In [14]:
conn = sqlite3.connect('patient_slim.sqlite')
cursor = conn.cursor()
# pull all data from the lab table except for the "key" column 
cursor.execute("SELECT * FROM demographics;")
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[column[0] for column in cursor.description])
conn.close()

In [15]:
# let's create a new sqlite database called patient_demonstration.sqlite
conn = sqlite3.connect('patient_demonstration.sqlite')
cursor = conn.cursor() 

In [16]:
# create a table called clinical_visits with the column names pulled above
# overwite the table if it already exists
cursor.execute('DROP TABLE IF EXISTS demographics;')
cursor.execute('''
CREATE TABLE demographics (
    PatientPKHash TEXT,
    MFLCode TEXT,
    FacilityName TEXT,
    County TEXT,
    SubCounty TEXT,
    PartnerName TEXT,
    AgencyName TEXT,
    Sex TEXT,
    MaritalStatus TEXT,
    EducationLevel TEXT,
    Occupation TEXT,
    OnIPT TEXT,
    AgeGroup TEXT,
    ARTOutcomeDescription TEXT,
    AsOfDate TEXT,
    LoadDate TEXT,
    StartARTDate TEXT,
    DOB TEXT,
    key TEXT
);
''')

# let's now populate the table with the rows variable that contains all the data from the visits table
cursor.executemany('''
INSERT INTO demographics (PatientPKHash, MFLCode, FacilityName, County, SubCounty, PartnerName, AgencyName, Sex, MaritalStatus, EducationLevel, Occupation, OnIPT, AgeGroup, ARTOutcomeDescription, AsOfDate, LoadDate, StartARTDate, DOB, key)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
''', rows)
conn.commit()

In [17]:
# now, add pharmacy table to the data dictionary
cursor.execute('''
INSERT INTO data_dictionary (table_name, column_name, description) VALUES
('demographics', 'PatientPKHash', 'Hashed patient identifier'),
('demographics', 'MFLCode', 'Code for the clinical site, same as SiteCode'),
('demographics', 'FacilityName', 'Name of the clinical facility'),
('demographics', 'County', 'County where the patient is located'),
('demographics', 'SubCounty', 'Sub-county where the patient is located'),
('demographics', 'PartnerName', 'Name of the implementing partner that manages the facility'),
('demographics', 'AgencyName', 'Name of the agency that supports the facility'),
('demographics', 'Sex', 'Sex of the patient. Valid values are male and female. Capitalization is not standardized so always set to lower case.'),
('demographics', 'MaritalStatus', 'Marital status of the patient. Valid values include married monogamous,
               married polygamous, single, divorced, widowed, cohabiting, separated. There are also some erroneous values 
               that should be ignored and treated as missing.'),
('demographics', 'EducationLevel', 'Education level of the patient. Valid values primary, secondary, tertiary, none. 
               there is a value for NULL that should be treated as missing.'),
('demographics', 'Occupation', 'Occupation of the patient. Valid values include farmer, trader, none (for unemployed),
               student, self employed, professional, employee, driver, and NULL that should be treated as missing.'),
('demographics', 'OnIPT', 'Indicates if the patient is on IPT. This is all null.'),
('demographics', 'AgeGroup', 'Age group of the patient. This is all null.'),
('demographics', 'ARTOutcomeDescription', 'Description of the ART outcome. Valid values include active, dead,
               loss to follow up, transferred out, undocumented loss, and lost in hmis.'),
('demographics', 'AsOfDate', 'Date as of which the data is reported'),
('demographics', 'LoadDate', 'Date when the data was loaded'),
('demographics', 'StartARTDate', 'Date when the patient started ART'),
('demographics', 'DOB', 'Date of birth of the patient'),
('demographics', 'key', 'Unique key for PatientPKHash and MFLCode combination');
''')
conn.commit()
conn.close()